# Assignment 2A

**Name**: Yash Barjatya                              </br>
**Roll No.**:  201138

In [11]:
# import
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random
import sys

In [4]:
'''
Represents a Stochastic Maze problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticMazeEnv(gym.Env):
    '''
    StochasticMazeEnv represents the Gym Environment for the Stochastic Maze environment
    States : [0,1,2,3,4,5,6,7,8,9,10,11]
    Actions : ["Left":0, "Up":1, "Right":2, "Down":3]
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=12,no_actions=4):
        '''
        Constructor for the StochasticMazeEnv class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 12
            no_actions : The no. of possible actions which is 4
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.actions_dict = {"L":0, "U":1, "R":2, "D":3}
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        return self.prob_dynamics

    
    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        index = np.random.choice(3,1,p=[0.8,0.1,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

In [5]:
env = StochasticMazeEnv()
env.reset()
num_states = env.nS
num_actions = env.nA

## Test Cases for checking the environment 

## Example: Random Policy

In [6]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    rand_action = np.random.choice(4,1)[0]  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(rand_action)
    
    total_reward += reward

    print("  ", rand_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   0 	   1 	 -0.01 	   False
   1 	   1 	 -0.01 	   False
   3 	   1 	 -0.01 	   False
   3 	   1 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 9
Final Reward: 0.92


### The random policy takes large number of steps to reach some terminal state which should be much higher than the number of the steps taken by a all 'Right' policy

## Example: Right Policy

In [7]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    right_action = 2  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


### The right policy most of the time reaches the goal state in just 3 steps which is expected

***

## Q1. Find an optimal policy to navigate the given environment using Policy Iteration 

In [89]:
# Initialization
policy = {0: 2, 1: 2, 2: 2, 4: 2, 6: 2, 8: 2, 9: 2, 10: 2, 11: 2}
policy_old = {}
value_f_old = {0: 1, 1: 1, 2: 1, 3: 0, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1, 10: 1, 11: 1}
value_f = value_f_old.copy()
# Policy iteration
loops_pei = 0  # Number of policy evaluation iteration loops
loops_pii = 0  # Number of policy improvement iteration loops
gamma =1 #discount_Factor
while policy_old != policy:
    loops_pii += 1
    # Policy Evaluation
    tolerance = 1e-6
    dif = 1
    while dif > tolerance:
        loops_pei += 1
        for curr_state in policy.keys():
            action = policy[curr_state]
            value_f[curr_state] = 0
            for next_state_info in env.prob_dynamics[curr_state][action]:
                prob,next_state, reward, _ = next_state_info
                value_f[curr_state] += prob * (reward + gamma*value_f_old[next_state])  # Bellman equation

        dif = np.sum(np.abs(np.array([value_f[k] - value_f_old[k] for k in policy.keys()])))
        value_f_old = value_f.copy()

    # Policy improvement
    policy_old = policy.copy()
    for curr_state in policy.keys():
        avail_actions = list(env.prob_dynamics[curr_state].keys())
        max_action_value = -sys.maxsize
        best_action = policy[curr_state]
        for action in avail_actions:
            action_value = 0
            for next_state_info in env.prob_dynamics[curr_state][action]:
                prob,next_state, reward, _ = next_state_info
                action_value += prob * (reward + gamma*value_f[next_state])
            if action_value > max_action_value:
                max_action_value = action_value
                best_action = action
        policy[curr_state] = best_action


# Convert policy from state indices to action labels
policy_labels = {0: 'left', 1: 'up', 2: 'right', 3: 'down'}
policy_policy_iteration = {f"s{i}": policy_labels[action] for i, action in policy.items()}
value_f_policy_iteration = {f"s{i}": val for i, val in value_f.items()}



In [90]:
print("Optimal policy using policy iteration:", policy_policy_iteration)
print("Value function using policy iteration:", value_f_policy_iteration)
print("Average number of policy evaluation iteration loops:", loops_pei/loops_pii)
print("Number of policy improvement iteration loops:", loops_pii)

Optimal policy using policy iteration: {'s0': 'right', 's1': 'right', 's2': 'right', 's4': 'up', 's6': 'left', 's8': 'up', 's9': 'left', 's10': 'left', 's11': 'down'}
Value function using policy iteration: {'s0': 0.9597242536350085, 's1': 0.9737867558093476, 's2': 0.9862867573277768, 's3': 0, 's4': 0.9472242513565996, 's5': 0, 's6': 0.896580826024145, 's7': 0, 's8': 0.933161748093996, 's9': 0.9206617446752353, 's10': 0.9068749716325266, 's11': 0.8068666069874402}
Average number of policy evaluation iteration loops: 100.75
Number of policy improvement iteration loops: 4


## Q2. Find an optimal policy to navigate the given environment using Value Iteration

In [91]:
# Initialization
policy = {0: 2, 1: 2, 2: 2, 4: 2, 6: 2, 8: 2, 9: 2, 10: 2, 11: 2}
policy_old = {}
value_f_old = {0: 1, 1: 1, 2: 1, 3: 0, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1, 10: 1, 11: 1}
value_f = value_f_old.copy()

# Value iteration
max_iteration =1000
tolerance = 1e-6
dif = 1
loops_vi = 0  # Number of value iteration loops
gamma=1 #discount_factor

while dif > tolerance and loops_vi<max_iteration:

    loops_vi += 1
    for curr_state in policy.keys():
        value_f[curr_state] = 0
        for action in range(len(env.prob_dynamics[curr_state])):
            action_value = 0
            for next_state_info in env.prob_dynamics[curr_state][action]:
                prob,next_state, reward, _ = next_state_info
                action_value += prob * (reward + gamma*value_f_old[next_state])  # Bellman equation
            if action_value > value_f[curr_state]:
                value_f[curr_state] = action_value

    dif = np.sum(np.abs(np.array([value_f[k] - value_f_old[k] for k in policy.keys()])))
    value_f_old = value_f.copy()

# Policy improvement
for curr_state in policy.keys():
    avail_actions = list(env.prob_dynamics[curr_state].keys())
    max_action_value = -sys.maxsize
    best_action = policy[curr_state]
    for action in avail_actions:
        action_value = 0
        for next_state_info in env.prob_dynamics[curr_state][action]:
            prob, next_state, reward, _ = next_state_info
            action_value += prob * (reward + gamma*value_f[next_state])
        if action_value > max_action_value:
            max_action_value = action_value
            best_action = action
    policy[curr_state] = best_action

# Convert policy from state indices to action labels
policy_labels = {0: 'left', 1: 'up', 2: 'right', 3: 'down'}
policy_value_iteration = {f"s{i}": policy_labels[action] for i, action in policy.items()}
value_f_value_iteration = {f"s{i}": val for i, val in value_f.items()}


In [92]:
print("Optimal policy using value iteration:", policy_value_iteration)
print("Value function using value iteration:", value_f_value_iteration)
print("Number of value iteration loops:", loops_vi)

Optimal policy using value iteration: {'s0': 'right', 's1': 'right', 's2': 'right', 's4': 'up', 's6': 'left', 's8': 'up', 's9': 'left', 's10': 'left', 's11': 'down'}
Value function using value iteration: {'s0': 0.9597242730305761, 's1': 0.9737867713955914, 's2': 0.9862867702538153, 's3': 0, 's4': 0.9472242747438153, 's5': 0, 's6': 0.8965809247091233, 's7': 0, 's8': 0.9331617771971151, 's9': 0.9206617797678368, 's10': 0.9068750213307932, 's11': 0.8068835635255157}
Number of value iteration loops: 102


## Q3. Compare PI and VI in terms of convergence. Is the policy obtained by both same?

### Answer: 

- Both algorithms, Policy Iteration (PI) and Value Iteration (VI), are guaranteed to converge to an optimal policy in the end.

- The optimal policy given by both methods is the same.

- In terms of convergence, Policy Iteration (PI) involves two main steps: policy evaluation and policy improvement. It iterates between these steps until the policy converges. However, it tends to converge relatively slowly as it alternates between evaluating and improving the policy in each iteration. Policy Iteration requires multiple iterations to converge, and in our case, it took 4 policy improvement loops, with an average of 100.75 policy evaluation iterations in each loop, resulting in a total of 100.75×4=403 iterations.

- On the other hand, Value Iteration (VI) directly iterates over the value function and updates it until it converges to the optimal value function. It typically converges faster than PI because it doesn't alternate between policy evaluation and improvement. VI converges after a smaller number of iterations compared to PI. In our case, it took 102 iterations for Value Iteration to converge.

- In summary, while both algorithms eventually converge to the optimal policy, Value Iteration tends to converge faster due to its direct iteration over the value function, whereas Policy Iteration alternates between policy evaluation and improvement, resulting in a slower convergence rate.